In [1]:
!pip uninstall -y fsspec datasets
!pip install fsspec==2023.6.0 datasets --no-cache-dir

Found existing installation: fsspec 2025.3.2
Uninstalling fsspec-2025.3.2:
  Successfully uninstalled fsspec-2025.3.2
Found existing installation: datasets 2.14.4
Uninstalling datasets-2.14.4:
  Successfully uninstalled datasets-2.14.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 73.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.6.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti

In [2]:
!pip install transformers datasets --quiet

In [3]:
import torch

In [4]:
from datasets import load_dataset
from transformers import GPT2Tokenizer

# Cargar el dataset IMDb (solo texto)
dataset = load_dataset("imdb", split="train")
texts = [x["text"] for x in dataset.select(range(2000))]  # Subconjunto para prueba rápida

# Tokenizador de GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 no tiene pad_token


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [5]:
from datasets import Dataset
import pandas as pd

# Convertir a Hugging Face Dataset
ds = Dataset.from_pandas(pd.DataFrame({"text": texts}))

# Tokenización para lenguaje
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = ds.map(tokenize_function, batched=True, remove_columns=["text"])


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:
!rm -rf gpt2-imdb
!rm -rf gpt2-finetuned


In [9]:
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,                     # ← ¡Faltaba esta línea!
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)

# Tokenizador
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # porque redefinimos pad_token

# Agrupador para MLM
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./gpt2-imdb",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    logging_steps=30,
    report_to="none",
    fp16=torch.cuda.is_available(),
    save_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()


<ipython-input-9-1983707695>:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
30,4.082000
60,3.868300
90,3.835500
120,3.832500
150,3.743100
180,3.701100
210,3.835000
240,3.776200
270,3.822800
300,3.643200


TrainOutput(global_step=1000, training_loss=3.7458515243530273, metrics={'train_runtime': 91.8142, 'train_samples_per_second': 21.783, 'train_steps_per_second': 10.892, 'total_flos': 130646016000000.0, 'train_loss': 3.7458515243530273, 'epoch': 1.0})

In [10]:
from transformers import pipeline

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "This movie was"
result = generator(prompt, max_new_tokens=100, temperature=0.9, top_k=50)
print(result[0]["generated_text"])


Device set to use cuda:0


This movie was so uninspired I could not enjoy it! The plot was not so much fun considering I am a fan of the "Crazy Little Bitty" genre but the dialog was so poorly put together. It was annoying, bad and annoying. It is very simple, but not what many people think of. It just is not well done. I felt very bad for having the opportunity to see the movie at the movie theater. I felt bad for having to watch this movie but I guess that was just
